In [32]:
import torch
import torch.nn as nn
from torch.nn import functional as F
device = 'cuda' if torch.cuda.is_available() else 'cpu'
print(device)
block_size = 8
batch_size = 4
max_iters = 10000
learning_rate = 3e-4
eval_iters = 250

cpu


In [33]:
with open('wizard_of_oz.txt','r', encoding = 'utf-8') as f:
    text = f.read()
chars = sorted(set(text))
vocab_size = len(chars)
print(chars)

['\n', ' ', '!', '"', '&', "'", '(', ')', '*', ',', '-', '.', '0', '1', '2', '3', '4', '5', '6', '7', '8', '9', ':', ';', '?', 'A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J', 'K', 'L', 'M', 'N', 'O', 'P', 'Q', 'R', 'S', 'T', 'U', 'V', 'W', 'X', 'Y', 'Z', '[', ']', '_', 'a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y', 'z', '\ufeff']


In [34]:
string_to_int = {ch:i for i,ch in enumerate(chars)}
int_to_str = {i:ch for i, ch in enumerate(chars) }
encode = lambda s: [string_to_int[c] for c in s]
decode = lambda l: ''.join([int_to_str[i] for i in l])
encoded_hello=encode('hello')
decoded_hello=decode(encoded_hello)

data = torch.tensor(encode(text), dtype=torch.long)

print(data[:100])

tensor([80,  1, 28, 39, 42, 39, 44, 32, 49,  1, 25, 38, 28,  1, 44, 32, 29,  1,
        47, 33, 50, 25, 42, 28,  1, 33, 38,  1, 39, 50,  0,  0,  1,  1, 26, 49,
         0,  0,  1,  1, 36, 11,  1, 30, 42, 25, 38, 35,  1, 26, 25, 45, 37,  0,
         0,  1,  1, 25, 45, 44, 32, 39, 42,  1, 39, 30,  1, 44, 32, 29,  1, 47,
        33, 50, 25, 42, 28,  1, 39, 30,  1, 39, 50,  9,  1, 44, 32, 29,  1, 36,
        25, 38, 28,  1, 39, 30,  1, 39, 50,  9])


In [35]:
n = int(0.8 * len(data))
train_data = data[:n]
val_data = data[n:]

In [36]:
block_size = 8
x = train_data[:block_size]
y = train_data[1: block_size +1]
print ('train_data: ' + str(train_data[:block_size+1]))
for i in range(block_size):
    context = x[:i+1]
    target=y[i]
    print('when input is', context, 'target is', target)

train_data: tensor([80,  1, 28, 39, 42, 39, 44, 32, 49])
when input is tensor([80]) target is tensor(1)
when input is tensor([80,  1]) target is tensor(28)
when input is tensor([80,  1, 28]) target is tensor(39)
when input is tensor([80,  1, 28, 39]) target is tensor(42)
when input is tensor([80,  1, 28, 39, 42]) target is tensor(39)
when input is tensor([80,  1, 28, 39, 42, 39]) target is tensor(44)
when input is tensor([80,  1, 28, 39, 42, 39, 44]) target is tensor(32)
when input is tensor([80,  1, 28, 39, 42, 39, 44, 32]) target is tensor(49)


In [37]:
def get_batch(split):
    data = train_data if split == 'train' else val_data
    ix = torch.randint(len(data) - block_size, (batch_size,))
    x=torch.stack([data[i:i+block_size] for i in ix])
    y = torch.stack([data[i+1:i+block_size+1] for i in ix])
    x,y = x.to(device), y.to(device)
    return x, y
x, y = get_batch('train')
print('inputs: ')
print(x)
print('targets: ')
print(y)

inputs: 
tensor([[67,  1, 73, 61, 58,  1, 76, 68],
        [73, 61, 71, 68, 74, 60, 61,  1],
        [65, 11,  1, 44, 61, 58, 67,  1],
        [74, 73,  1, 54, 72,  1, 76, 62]])
targets: 
tensor([[ 1, 73, 61, 58,  1, 76, 68, 68],
        [61, 71, 68, 74, 60, 61,  1, 62],
        [11,  1, 44, 61, 58, 67,  1, 61],
        [73,  1, 54, 72,  1, 76, 62, 57]])


In [38]:
@torch.no_grad()
def estimate_loss():
    out = {}
    model.eval()
    for split in ['train', 'val']:
        losses = torch.zeros(eval_iters)
        for k in range(eval_iters):
            X, Y = get_batch(split)
            logits, loss = model(X, Y)
            losses[k] = loss.item()
        out[split] = losses.mean()
    model.train()
    return out

In [39]:
class BigramLanguageModel(nn.Module):
    def __init__(self, vocab_size):
        super().__init__()
        self.token_embedding_table = nn.Embedding(vocab_size, vocab_size)

    def forward(self, index, targets = None):
        logits = self.token_embedding_table(index)

        if targets is None:
            loss = None
        else:
            B, T, C = logits.shape
            logits = logits.view(B*T, C)
            targets = targets.view(B*T)
            loss = F.cross_entropy(logits, targets)
            
        return logits, loss

    def generate(self, index, max_new_tokens):
        # index is (B, T) array of indices in the current context
        for _ in range(max_new_tokens):
            #get predictions
            logits, loss = self.forward(index)
            # focus only on the last time step
            logits = logits[:, -1, :] # becomes (B,C)
            # apply softmax to get probabilities
            probs = F.softmax(logits, dim=-1) # (B, C)
            # sample from the distribution
            index_next = torch.multinomial(probs, num_samples=1) # (B, 1)
            # append sampled index to the running sequence
            index = torch.cat((index, index_next), dim=-1) # (B, T+1)
        return index

model = BigramLanguageModel(vocab_size)
m = model.to(device)

context = torch.zeros((1,1), dtype=torch.long, device=device)
generated_chars = decode(m.generate(context, max_new_tokens=500)[0].tolist())
print(generated_chars)

        


_sb7J:LLsS,b8agScR;00zcokPtY:_BGHX,"RGsVKBTu:ppM) w7Q0a9][gkx.ljw2pI)j['3xSI(tUw_*xXmj5:O!7;M]GQ]
9VujF-MRT7&PkF
?n'39Z_j]Z*xhh ;jK-L1IERX9RRTvhp(Zu(IrK(eCjPRw7gse3LT_:pjaZ*h"J b1[NRJ,-tP4m?
9sR!!R7UkSl:[pAh
(4Lm4Ue55PerZ0)'8Cfyee3zDjpmfJlnxavEdS_a;V-OycL7[&8h1T6qRuW.P5dN!qmMqJ0BpdpV:OIE[mV-(MFR?cc-WHfKsD[Zutlnm;NU﻿Nag1m&MV-BtE[
?l:x6"Fwz-WIn﻿l??lr?qNB&
2M)drhpIQ(0qrLPD0GVUhum1lo_n0&7gTt.tibD1l﻿o'&?lx2*f57j[w"ZKhFyp;!RJ﻿;76qxW:Dz[Yor*QF]fI0V-c'hNQw[zH?lhw1JGRTucgjJ,;SxND1D,Ra wi6Avye3rxqj Tb*XCg


In [53]:
# Create a PyTorch optimizer
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

for iter in range(max_iters):
    if iter % eval_iters == 0:
        losses = estimate_loss()
        print(f'step: {iter}, train loss: {losses["train"]:.4f}, val loss: {losses["val"]:.4f}')

    #sample a batch of data
    xb, yb = get_batch('train')

    # evaluate the loss
    logits, loss = model.forward(xb, yb)
    optimizer.zero_grad(set_to_none = True)
    loss.backward()
    optimizer.step()
print(loss.item())

step: 0, train loss: 2.5300, val loss: 2.5452
step: 250, train loss: 2.5067, val loss: 2.5511
step: 500, train loss: 2.5244, val loss: 2.5642
step: 750, train loss: 2.5449, val loss: 2.5733
step: 1000, train loss: 2.5072, val loss: 2.5671
step: 1250, train loss: 2.5028, val loss: 2.5427
step: 1500, train loss: 2.4630, val loss: 2.5329
step: 1750, train loss: 2.5039, val loss: 2.5811
step: 2000, train loss: 2.5055, val loss: 2.5334
step: 2250, train loss: 2.4948, val loss: 2.5622
step: 2500, train loss: 2.5087, val loss: 2.5404
step: 2750, train loss: 2.4940, val loss: 2.5371
step: 3000, train loss: 2.5126, val loss: 2.5335
step: 3250, train loss: 2.5070, val loss: 2.5443
step: 3500, train loss: 2.4784, val loss: 2.5385
step: 3750, train loss: 2.4817, val loss: 2.5325
step: 4000, train loss: 2.4900, val loss: 2.5456
step: 4250, train loss: 2.4950, val loss: 2.5709
step: 4500, train loss: 2.4914, val loss: 2.5080
step: 4750, train loss: 2.4854, val loss: 2.5308
step: 5000, train loss: 2.

In [54]:
context = torch.zeros((1, 1), dtype=torch.long, device=device)
generated_chars = decode(m.generate(context, max_new_tokens=500)[0].tolist())
print(generated_chars)


Tho grfowin'de ilo w f
tusprey s, the trems fl theableskertin wey aiz*whe od gith, ind thakincu

"C9h  ore hee un, ofur, rindidnoncathiead satulethe mad tly.
"DXcker m
tizma nount.
"To wl und thed wains plwandrul?F cond hyitlknsteat  sea t  slllyerefooryontureruborouram.]-ayoroup, inthe t ghom re sieng
d anhe bow hin sederaloung, hy t,"YBlise ore the, t ty.
e t thehean'tal he med
k byo
FRulyord ponn e it-BORDotint a the ow om."HTR!RDugr;Vq, be s her wean houlilo whery ry therin taid
llor?Afrath,
